# ColabGCNdesign_beta
This notebook is for running Dr. Shintaro Minami's [GCNdesign](https://github.com/ShintaroMinami/GCNdesign) on Google Colaboratory.

In [ ]:
#@title 0. Install software
#@markdown Please run this cell by pressing the ▶ _Play_ button on the left.

#@markdown PyRosetta and GCNdesign will be installed.

#@markdown PyRosetta is ~1.5 GB. It takes several tens of minutes to install PyRosetta.

#@markdown  Enter your user ID and password to download PyRosetta.
#@markdown  You can obtain a license for PyRosetta at the following link https://els2.comotion.uw.edu/product/pyrosetta

import os, sys, time
USER = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
!mkdir /content/PyRosetta
!cd /content/PyRosetta && wget -O _PyRosetta_.tar.bz2 --no-check-certificate --user $USER --password $PASSWORD http://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.MinSizeRel.python37.linux/PyRosetta4.MinSizeRel.python37.linux.release-316.tar.bz2
!tar xjf /content/PyRosetta/_PyRosetta_.tar.bz2 -C /content/PyRosetta --strip-components=1
pyrosetta_distr_path = '/content' + '/PyRosetta/_PyRosetta_.tar.bz2'
!cd /content/PyRosetta/setup && python setup.py install

for e in os.listdir('/content/PyRosetta/'):
    sys.path.append('/content/PyRosetta/' + e)

!pip install gcndesign
!pip install py3Dmol

In [ ]:
#@title 1. Autodesign
#! /usr/bin/env python

import os
from os import path
import sys
import argparse
from google.colab import files

# argument parser
pdbcode = "6WMK" #@param {type:"string"}
#@markdown - `pdbcode` Enter the PDB ID of the protein structure you want to design. ( e.g. 1qys ) 
#@markdown - **<font color="red">To input your own PDB file, run the cell without writing anything in this field and then select the file.</font>**
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    import pyrosetta
    pyrosetta.init()
    pyrosetta.toolbox.cleanATOM(f"{pdb_code}.pdb")
    clean_pdb=f"{pdb_code}.clean.pdb"
    pose_in=pyrosetta.pose_from_pdb(clean_pdb)
    chains = ["A","B"] #@param {type:"raw"}
    extract_chain="".join(chains)
    #@markdown - `chains` If you filled in the "pdbcode" field above, please enter the name of the Chain you wish to design. ( e.g. ["A"] or ["A","B","C"])
    r=[]
    for res in pose_in.residues:
        num=res.seqpos()
        chainletter=pose_in.pdb_info().chain(num)
        print(num, chainletter)
        for chain in chains:
            if chainletter == chain:
                r.append(num)
    pose=pyrosetta.Pose(pose_in, r[0], r[-1])
    print(r)
    print(r[0])
    print(r[-1])
    pose.dump_pdb(f"{pdb_code}_{extract_chain}.clean.pdb")
    return f"{pdb_code}_{extract_chain}.clean.pdb"

#@title Select a PDB file, enter options, and then press _Play_ ▶ button to design.

pdb = get_pdb(pdbcode)

opts=["-ignore_unrecognized_res 1", "-ex1", "-ex2aro"]
# ligand input
ligand =  "" #@param {type:"string"}
#@markdown - `ligand` ligand/substrate parameter file. ( e.g. default: None )
#@markdown - If your PDB file includes a ligand, you need a params file for the ligand. Also, the input PDB file must also contain the coordinates of the substrate.
#@markdown - parameter file should be created using Rosetta's "/Rosetta/main/source/scripts/python/public/molfile_to_params.pyl".
#@markdown - See the following link for more information　https://www.rosettacommons.org/demos/latest/tutorials/prepare_ligand/prepare_ligand_tutorial
if ligand:
    opts.append("-extra_res_fa "+ligand)

symmetry_definition_file =  "" #@param {type:"string"}
#@markdown - `symmetry_definition_file` If you want to do symmetry design, specify the symmetry definition file for Rosetta Design. ( e.g. default: None )
#@markdown - Symmetry definition file should be created using Rosetta's "/Rosetta/main/source/src/apps/public/symmetry/make_symmdef_file.pl".
#@markdown - See the following link for more information　https://www.rosettacommons.org/docs/latest/application_documentation/utilities/make-symmdef-file

nstruct = 1 #@param {type:"integer"}
#@markdown - `nstruct` Number of structures output ( e.g. 10 )
prefix = "autodes" #@param {type:"string"}
#@markdown - `prefix` Prefix for output PDB files. ( e.g. autodes ) 
prob_cut = 0.8 #@param {type:"number"}
#@markdown - `prob_cut` Probability cutoff. ( e.g. 0.8 )
scorefxn = "ref2015" #@param {type:"string"}
#@markdown - `scorefxn` Rosetta score function. ( e.g. ref2015 )
keep = [] #@param {type:"raw"}
#@markdown - `keep` Residue numbers for keeping the initial amino-acid type. ( e.g. [] or ["1A-10A","15B","30B","32C-40C","@D"] @ represents all residues in the chain )
unused =  ["C", "H", "W"] #@param {type:"raw"}
#@markdown - `unused` Residue types not to be used in design sequences. ( e.g. None or ["C",  "H", "W"] )
include_init_restype = True #@param {type:"boolean"}
#@markdown - `include_init_restype` Include the initial residue type.
fastdesign_iterations = 1 #@param {type:"integer"}
#@markdown - `fastdesign_iterations` "standard_repeats" for Rosetta FastDesign. ( e.g. 3 )
paramin = None #@param {type:"raw"}
#@markdown - `paramin` NN parameter file. ( e.g. default: None )

# pyrosetta
import pyrosetta
pyrosetta.init(opts)
scorefxn = pyrosetta.create_score_function(scorefxn)

# gcndesign predictor
#dir_script = path.dirname(path.realpath(__file__))
#sys.path.append(dir_script+'/../')
from gcndesign.predictor import Predictor
predictor = Predictor(param=paramin)

# pdb input
pose_in = pyrosetta.pose_from_pdb(pdb)
if symmetry_definition_file:
    from pyrosetta.rosetta.protocols.symmetry import SetupForSymmetryMover
    #DetectSymmetry().apply(pose_in)
    SetupForSymmetryMover(symmetry_definition_file).apply(pose_in)

## Setup TaskFactory
taskf = pyrosetta.rosetta.core.pack.task.TaskFactory()
taskf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
if include_init_restype:
    taskf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())

# resfile task-operation
from gcndesign.resfile import fix_native_resfile, expand_nums
resfile = predictor.make_resfile(pdb=pdb, prob_cut=prob_cut, unused=unused)
resfile = fix_native_resfile(resfile, resnums=expand_nums(keep))
readresfile = pyrosetta.rosetta.core.pack.task.operation.ReadResfile()
readresfile.set_cached_resfile(resfile)

# add readresfile to taskfactory
taskf.push_back(readresfile)

## Check TaskFactory Setting
packer_task = taskf.create_task_and_apply_taskoperations(pose_in)

## Setup MoveMapFactory
movemapf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
movemapf.all_bb(setting=True)
movemapf.all_chi(setting=True)
movemapf.all_jumps(setting=True)

## Check Setting
#display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
#display_pose.tasks(taskf)
#display_pose.movemap_factory(movemapf)
#display_pose.apply(pose)

## Mover Setting
fastdesign = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, standard_repeats=fastdesign_iterations)
fastdesign.set_task_factory(taskf)
fastdesign.set_movemap_factory(movemapf)

## Apply
for i in range(nstruct):
    pose = pose_in.clone()
    fastdesign.apply(pose)
    file_out = '{:s}-{:03d}.pdb'.format(prefix, i+1)
    pose.dump_pdb(file_out)

In [ ]:
#@title 2. View 3D Structure {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt

color = "rainbow" #@param ["rainbow", "chain"]
show_sidechains = True #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

#pdb_file = glob.glob(pdb_filename)
pdb_file = "autodes-001.pdb"  #@param {type:"string"}
def show_pdb(color="rainbow", show_sidechains=False, show_mainchains=False):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
    view.addModel(open(pdb_file,'r').read(), 'pdb')
    if color == "rainbow":
        view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
        import pyrosetta
        pyrosetta.init()
        pose_input=pyrosetta.pose_from_pdb(pdb_file)
        chains = pose_input.pdb_info().num_chains() if pose_input.pdb_info().num_chains() >=2 else 1
        for n,chain,color in zip(range(chains),list("ABCDEFGH"), ["lime","cyan","magenta","yellow","salmon","white","blue","orange"]):
            view.setStyle({'chain':chain},{'cartoon': {'color':color}})
    if show_sidechains:
        BB = ['C','O','N']
        view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]}, {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]}, {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]}, {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
    if show_mainchains:
        BB = ['C','O','N','CA']
        view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.zoomTo()
    return view


show_pdb(color, show_sidechains, show_mainchains).show()

# License
The source code of ColabGCNdesign is licensed under [MIT](https://github.com/naokob/ColabGCNdesign/blob/main/LICENSE).

This notebook uses the GCNdesign source code and its parameters licensed under [MIT](https://github.com/ShintaroMinami/GCNdesign/blob/master/LICENSE).